In [1]:
# Data manipulation
from sklearn.preprocessing import LabelEncoder
from scipy.stats import loguniform
import numpy as np
import pandas as pd
import os

# Data Visualazation
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()
%matplotlib inline

#consistent sized plots
from pylab import rcParams
rcParams['figure.figsize'] = 12,5
rcParams['xtick.labelsize'] = 12
rcParams['ytick.labelsize'] = 12
rcParams['axes.titlesize'] = 14
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

#handle unwanted warnings
import warnings
warnings.filterwarnings('ignore')

# Machine learning
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from lightgbm import LGBMModel,LGBMClassifier
import catboost as cb
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import VotingClassifier

# Model Evaluations
import sklearn as sk
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score

#no constraint on the data columns to be displayed in jupyter notebook
pd.options.display.max_columns = None

# Handle Date & Time
from datetime import datetime

# Handle zip files
import zipfile

# Print module versions for reproducibility
print('CatBoost version {}'.format(cb.__version__))
print('NumPy version {}'.format(np.__version__))
print('Pandas version {}'.format(pd.__version__))
print('seaborn version {}'.format(sns.__version__))
print('sklearn version {}'.format(sk.__version__))
print('xgboost version {}'.format(xgb.__version__))
print('lightgbm version {}'.format(lgb.__version__))

# User Defined Functions

# a) BoxPlot:- To check Outliers
def Create_BoxPlotS(DataFrame, Features, Rows, Columns):
    fig=plt.figure(figsize=(20,20))
    for i, Feature in enumerate(Features[0:]):
        ax=fig.add_subplot(Rows,Columns,i+1)
        sns.boxplot(DataFrame[Feature],ax=ax)   
    fig.tight_layout()  
    plt.show()
    
# b) DistPlot:- To check Normality
def Create_DistPlots(DataFrame, Features, Rows, Columns):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(Features[0:]):
        ax=fig.add_subplot(Rows,Columns,i+1)
        sns.distplot(DataFrame[feature],ax=ax)    
    fig.tight_layout()  
    plt.show()

# c) Univariate Analysis of Catagorical Variables
def Univariate_Cat_Features(DataFrame, Features, Rows, Columns):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(Features[0:]):
        ax=fig.add_subplot(Rows,Columns,i+1)
        sns.countplot(DataFrame[feature],ax=ax)    
    fig.tight_layout()  
    plt.show()
    
# d) Scatter Plot:- Bivariate Analysis between Target Variable & Independent Variables
# Note:- Pass Target_Variable in Single Quote in this function
def Create_ScatterPlots(DataFrame, Features, Target_Variable, Rows, Columns):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(Features[0:]):
        ax=fig.add_subplot(Rows,Columns,i+1)
        sns.scatterplot(DataFrame[feature],DataFrame[Target_Variable],ax=ax)  
    fig.tight_layout()  
    plt.show()

# e) Joint Plot:- Bivariate Analysis between Target Variable & Independent Variables
# Note:- Pass Target_Variable in Single Quote in this function
def Create_JointPlots(DataFrame, Features, Target_Variable):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(Features[0:]):
        sns.jointplot(y= Target_Variable, x= DataFrame[feature], data = DataFrame,kind='reg')
    fig.tight_layout()  
    plt.show()

# f) Check Feature wise Outliers   
def Check_Outliers(DataFrame, Column_List):
    for column in Column_List:    
        q1 = DataFrame[column].quantile(0.25)    # First Quartile
        q3 = DataFrame[column].quantile(0.75)    # Third Quartile
        IQR = q3 - q1                            # Inter Quartile Range
        llimit = q1 - 1.5*IQR                    # Lower Limit
        ulimit = q3 + 1.5*IQR                    # Upper Limit
        outliers = DataFrame[(DataFrame[column] < llimit) | (DataFrame[column] > ulimit)]
        print('\nNumber of outliers in "' + column + '" :' + str(len(outliers)))
        print('\nLower Limit in "' + column + '" :',llimit)
        print('\nUpper Limit in "' + column + '" :',ulimit)
        print('\nInter Quartile Range of "' + column + '" :',IQR,"\n")

# g) Label Encoding   
def Label_Encoder(DataFrame):
    le = LabelEncoder()
    for col in DataFrame.columns:
        if(DataFrame[col].dtype == 'object'):
            DataFrame.loc[:,col] = le.fit_transform(DataFrame.loc[:,col])
            
            
train = pd.read_csv (r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\Train.csv',encoding='latin-1')
test = pd.read_csv (r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\Test.csv',encoding='latin-1')
test1 = pd.read_csv (r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\Test.csv',encoding='latin-1')
submission = pd.read_csv (r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\Sample_Submission.csv',encoding='latin-1')

print("Checking Null entries & their Percentage in Train Data:-\n\n")
missing_val_train=pd.DataFrame(zip(train.isnull().sum(),train.isnull().sum()*100/len(train)),columns=['Missing Values','Percentage Missing Values'],index=train.columns)
print(missing_val_train)

print("Checking Null entries & their Percentage in Test Data:-\n\n")
missing_val_test=pd.DataFrame(zip(test.isnull().sum(),test.isnull().sum()*100/len(test)),columns=['Missing Values','Percentage Missing Values'],index=test.columns)
missing_val_test

CatBoost version 1.2
NumPy version 1.21.5
Pandas version 1.4.2
seaborn version 0.11.2
sklearn version 1.0.2
xgboost version 1.7.6
lightgbm version 3.3.5
Checking Null entries & their Percentage in Train Data:-


                 Missing Values  Percentage Missing Values
ID                            0                   0.000000
DEVICETYPE                    0                   0.000000
PLATFORM_ID                   0                   0.000000
BIDREQUESTIP                  0                   0.000000
USERPLATFORMUID               4                   0.003511
USERCITY                   6359                   5.581154
USERZIPCODE                4592                   4.030297
USERAGENT                     2                   0.001755
PLATFORMTYPE                  0                   0.000000
CHANNELTYPE                   0                   0.000000
URL                           0                   0.000000
KEYWORDS                      0                   0.000000
TAXONOMY             

Missing Values  Percentage Missing Values
ID                            0                   0.000000
DEVICETYPE                    0                   0.000000
PLATFORM_ID                   0                   0.000000
BIDREQUESTIP                  0                   0.000000
USERPLATFORMUID               0                   0.000000
USERCITY                   1559                   5.471519
USERZIPCODE                1115                   3.913242
USERAGENT                     1                   0.003510
PLATFORMTYPE                  0                   0.000000
CHANNELTYPE                   0                   0.000000
URL                           0                   0.000000
KEYWORDS                      0                   0.000000

In [2]:

train['USERCITY'] = train['USERCITY'].fillna(train['USERCITY'].mode()[0],inplace=True)
test['USERCITY'] = test['USERCITY'].fillna(test['USERCITY'].mode()[0],inplace=True)
train['USERZIPCODE'] = train['USERZIPCODE'].fillna(train['USERZIPCODE'].mode()[0],inplace=True)
test['USERZIPCODE'] = test['USERZIPCODE'].fillna(test['USERZIPCODE'].mode()[0],inplace=True)
train['USERAGENT'] = train['USERAGENT'].fillna(train['USERAGENT'].mode()[0],inplace=True)
test['USERAGENT'] = test['USERAGENT'].fillna(test['USERAGENT'].mode()[0],inplace=True)

train.loc[(train['IS_HCP'].isnull()),'IS_HCP'] = 0

In [3]:
Label_Encoder(train)
Label_Encoder(test)

In [4]:
train = train.drop(['TAXONOMY'],axis=1)


In [5]:
X = train.drop(['IS_HCP'],axis=1)
y = train.IS_HCP
X_train, X_test, y_train, y_test = train_test_split(X,y, shuffle = True)

In [ ]:
model5 = LogisticRegression(random_state=42) #46.4611
model5.fit(X_train,y_train) 
print('Accuracy of Logistic Regression on training set: {:.2f}'
     .format(model5.score(X_train, y_train)))

print('Accuracy of Logistic Regression on test set: {:.2f}'
     .format(model5.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model5.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred5 = model5.predict(x); 
y_pred = y_pred5

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M5.csv',index=False)
print("Process Completed")

In [6]:
model5 = LogisticRegression(random_state=42)
solvers = ['newton-cg', 'sag', 'lbfgs']
penalty = ['l2']
C = [0.01, 0.1, 1.0, 10, 100]
param = dict(solver=solvers, penalty=penalty, C=C)
random_search = RandomizedSearchCV(estimator=model5, param_distributions=param, cv=5, scoring='accuracy')
random_result = random_search.fit(X_train, y_train)
print('-------------'*9)
print("Best Parameters after Random Search:- \n",random_result.best_params_)
print('-------------'*9)
print("Best Score:- \n",random_result.best_score_)

---------------------------------------------------------------------------------------------------------------------
Best Parameters after Random Search:- 
 {'solver': 'newton-cg', 'penalty': 'l2', 'C': 0.1}
---------------------------------------------------------------------------------------------------------------------
Best Score:- 
 0.7297547458671219


In [7]:
model55 = LogisticRegression(random_state=42
                            ,solver = 'newton-cg', penalty = 'l2'
                             , C = 0.1) #
model55.fit(X_train,y_train) 
print('Accuracy of Logistic Regression on training set: {:.2f}'
     .format(model55.score(X_train, y_train)))

print('Accuracy of Logistic Regression on test set: {:.2f}'
     .format(model55.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model55.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred55 = model55.predict(x); 
y_pred = y_pred55

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M55.csv',index=False)
print("Process Completed")

Accuracy of Logistic Regression on training set: 0.73
Accuracy of Logistic Regression on test set: 0.72
---------------------------------------------------------------------------------------------------------------------
Classification Report of the training data:

               precision    recall  f1-score   support

         0.0       0.73      0.97      0.83     19807
         1.0       0.68      0.17      0.27      8678

    accuracy                           0.72     28485
   macro avg       0.71      0.57      0.55     28485
weighted avg       0.71      0.72      0.66     28485
 

<class 'pandas.core.frame.DataFrame'>
Process Completed


In [ ]:
model1 = RandomForestClassifier(random_state=0) #43.0604
model1.fit(X_train,y_train) 
print('Accuracy of Random Forest Classifier on training set: {:.2f}'
     .format(model1.score(X_train, y_train)))

print('Accuracy of Random Forest Classifier on test set: {:.2f}'
     .format(model1.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model1.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred1 = model1.predict(x); 
y_pred = y_pred1

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M1.csv',index=False)
print("Process Completed")

In [11]:
# Hyper Version
model11 = RandomForestClassifier(random_state=0
                               ,n_estimators = 50, min_samples_split = 2
                                ,min_samples_leaf = 3, max_features = 'auto'
                                , max_depth = 10, bootstrap =  False) #
model11.fit(X_train,y_train) 
print('Accuracy of Random Forest Classifier on training set: {:.2f}'
     .format(model11.score(X_train, y_train)))

print('Accuracy of Random Forest Classifier on test set: {:.2f}'
     .format(model11.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model11.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred11 = model11.predict(x); 
y_pred = y_pred11

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M11.csv',index=False)
print("Process Completed")

Accuracy of Random Forest Classifier on training set: 0.89
Accuracy of Random Forest Classifier on test set: 0.88
---------------------------------------------------------------------------------------------------------------------
Classification Report of the training data:

               precision    recall  f1-score   support

         0.0       0.85      1.00      0.92     19807
         1.0       0.99      0.60      0.75      8678

    accuracy                           0.88     28485
   macro avg       0.92      0.80      0.83     28485
weighted avg       0.89      0.88      0.87     28485
 

<class 'pandas.core.frame.DataFrame'>
Process Completed


In [ ]:
model2 = LGBMClassifier(random_state=0) #50.0098
model2.fit(X_train,y_train) 
print('Accuracy of LGBM Classifier on training set: {:.2f}'
     .format(model2.score(X_train, y_train)))

print('Accuracy of LGBM Classifier on test set: {:.2f}'
     .format(model2.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model2.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred2 = model2.predict(x); 
y_pred = y_pred2

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M2.csv',index=False)
print("Process Completed")

In [ ]:
# Hyper Version
model22 = LGBMClassifier(random_state=42,reg_alpha = 2,num_leaves = 50
                         , n_estimators = 300, min_child_samples = 40
                         , max_depth = -1, learning_rate = 0.3, colsample_bytree = 0.5
                         , boosting_type = 'dart') #51.3348
model22.fit(X_train,y_train) 
print('Accuracy of LGBM Classifier on training set: {:.2f}'
     .format(model22.score(X_train, y_train)))

print('Accuracy of LGBM Classifier on test set: {:.2f}'
     .format(model22.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model22.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred22 = model22.predict(x); 
y_pred = y_pred22

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M22.csv',index=False)
print("Process Completed")

In [ ]:
model3 = CatBoostClassifier(random_state=0, logging_level='Silent') #44.6453
model3.fit(X_train,y_train) 
print('Accuracy of CatBoost Classifier on training set: {:.2f}'
     .format(model3.score(X_train, y_train)))

print('Accuracy of CatBoost Classifier on test set: {:.2f}'
     .format(model3.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model3.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred3 = model3.predict(x); 
y_pred = y_pred3

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M3.csv',index=False)
print("Process Completed")

In [13]:
model33 = CatBoostClassifier(random_state=0
                            , depth = 10, iterations = 100, learning_rate = 0.04,logging_level='Silent') #
model33.fit(X_train,y_train) 
print('Accuracy of CatBoost Classifier on training set: {:.2f}'
     .format(model33.score(X_train, y_train)))

print('Accuracy of CatBoost Classifier on test set: {:.2f}'
     .format(model33.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model33.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred33 = model33.predict(x); 
y_pred = y_pred33

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M33.csv',index=False)
print("Process Completed")

Accuracy of CatBoost Classifier on training set: 0.90
Accuracy of CatBoost Classifier on test set: 0.90
---------------------------------------------------------------------------------------------------------------------
Classification Report of the training data:

               precision    recall  f1-score   support

         0.0       0.89      0.98      0.93     19807
         1.0       0.95      0.71      0.81      8678

    accuracy                           0.90     28485
   macro avg       0.92      0.85      0.87     28485
weighted avg       0.90      0.90      0.89     28485
 

<class 'pandas.core.frame.DataFrame'>
Process Completed


In [ ]:
model4 = XGBClassifier(random_state=0) #48.87

model4.fit(X_train,y_train) 
print('Accuracy of XGB Classifier on training set: {:.2f}'
     .format(model4.score(X_train, y_train)))

print('Accuracy of XGB Classifier on test set: {:.2f}'
     .format(model4.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model4.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred4 = model4.predict(x); 
y_pred = y_pred4

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M4.csv',index=False)
print("Process Completed")

In [ ]:
# Hyper Version
model44 = XGBClassifier(random_state=0
                       ,subsample = 0.8999999999999999, num_class = 10,n_estimators = 500
                       ,min_child_weight = 3, max_depth = 15, learning_rate = 0.4
                       ,gamma = 0.2,colsample_bytree = 0.8999999999999999,colsample_bylevel = 0.6) #

model44.fit(X_train,y_train) 
print('Accuracy of XGB Classifier on training set: {:.2f}'
     .format(model44.score(X_train, y_train)))

print('Accuracy of XGB Classifier on test set: {:.2f}'
     .format(model44.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model44.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred44 = model44.predict(x); 
y_pred = y_pred44

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M44.csv',index=False)
print("Process Completed")

In [ ]:
model6 = KNeighborsClassifier(n_jobs= -1) #48.538
model6.fit(X_train,y_train) 
print('Accuracy of KNeighbors Classifier on training set: {:.2f}'
     .format(model6.score(X_train, y_train)))

print('Accuracy of KNeighbors Classifier on test set: {:.2f}'
     .format(model6.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model6.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred6 = model6.predict(x); 
y_pred = y_pred6

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M6.csv',index=False)
print("Process Completed")

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
model7 = make_pipeline(StandardScaler(),LinearSVC()) #46.8387

model7.fit(X_train,y_train) 
print('Accuracy of Linear SVC on training set: {:.2f}'
     .format(model7.score(X_train, y_train)))

print('Accuracy of Linear SVC on test set: {:.2f}'
     .format(model7.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model7.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred7 = model7.predict(x); 
y_pred = y_pred7

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M7.csv',index=False)
print("Process Completed")

In [ ]:
model8 = make_pipeline(StandardScaler(),SGDClassifier()) #45.5557

model8.fit(X_train,y_train) 
print('Accuracy of SGD Classifier on training set: {:.2f}'
     .format(model8.score(X_train, y_train)))

print('Accuracy of SGD Classifier on test set: {:.2f}'
     .format(model8.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model8.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred8 = model8.predict(x); 
y_pred = y_pred8

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M8.csv',index=False)
print("Process Completed")

In [ ]:
mm = make_pipeline(StandardScaler(),LinearSVC()) #47.341
model9 = CalibratedClassifierCV(mm)

model9.fit(X_train,y_train) 
print('Accuracy of Linear SVC on training set: {:.2f}'
     .format(model9.score(X_train, y_train)))

print('Accuracy of Linear SVC on test set: {:.2f}'
     .format(model9.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model9.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred9 = model9.predict(x); 
y_pred = y_pred9

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M9.csv',index=False)
print("Process Completed")

In [ ]:
ab = make_pipeline(StandardScaler(),SGDClassifier(loss = 'hinge')) #45.5708
model10 = CalibratedClassifierCV(ab)

model10.fit(X_train,y_train) 
print('Accuracy of SGD Classifier on training set: {:.2f}'
     .format(model10.score(X_train, y_train)))

print('Accuracy of SGD Classifier on test set: {:.2f}'
     .format(model10.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model10.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred10 = model10.predict(x); 
y_pred = y_pred10

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M10.csv',index=False)
print("Process Completed")